In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D,Conv2DTranspose,Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import os


In [77]:
def conv_block(x, n_base, batch_normalization):
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def downsample_block(x, n_base, batch_normalization, dropout):
    f = conv_block(x, n_base, batch_normalization)
    p = layers.MaxPool2D(pool_size = (2,2))(f)
    if(dropout):
        p = layers.Dropout(0.2)(p)
        
    return f, p

def upsample_block(x, f, n_base, batch_normalization, dropout):
    
    x = Conv2DTranspose(filters=n_base, kernel_size=(2,2), 
                         strides=(2,2),padding='same')(x)
    x = Concatenate()([x,f])
    if(dropout):
        x = layers.Dropout(0.2)(x)
    x = conv_block(x, n_base, batch_normalization)
        
    return x

In [78]:
def get_unet(img_w, img_h, img_ch, n_base, LR, batch_normalization, dropout):
    
#     ## Parameters
#     n_base = 8
#     LR = 1e-4
#     batch_normalization = True
#     dropout = False
    
    
    ## Encoder part
#     model = Sequential()
    inputs = layers.Input((img_w, img_h, img_ch))
    
    f1, p1 = downsample_block(inputs, n_base, batch_normalization, dropout)
    f2, p2 = downsample_block(p1, n_base*2, batch_normalization, dropout)
    f3, p3 = downsample_block(p2, n_base*4, batch_normalization, dropout)
    f4, p4 = downsample_block(p3, n_base*8, batch_normalization, dropout)
    
    
    ## Bottleneck
    bottleneck = conv_block(p4, n_base*16, batch_normalization)
    
    ## Decoder part
    p5 = upsample_block(bottleneck, f4, n_base*8, batch_normalization, dropout)
    p6 = upsample_block(p5, f3, n_base*4, batch_normalization, dropout)
    p7 = upsample_block(p6, f2, n_base*2, batch_normalization, dropout)
    p8 = upsample_block(p7, f1, n_base, batch_normalization, dropout)

    
    ## 1 Convo layer
    p9 = Conv2D(filters=1, kernel_size=(1,1), 
                            padding='same')(p8)
    outputs = Activation('sigmoid')(p9)
    

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
#     model = model(inputs = inputs, outputs = outputs)
    model.summary()
    
    return model


In [81]:
from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [145]:
## Task1a) Lung segmentation in chest X-ray images:
from random import shuffle
from skimage.io import imread
from skimage.transform import resize

n_base =16
LR = 1e-4
batch_normalization = True
dropout = True
epochs = 50
batch_size = 8

img_w, img_h = 240,240
img_ch = 1

model = get_unet(img_w, img_h, img_ch, n_base, LR, 
                 batch_normalization, dropout)

## BCE Parameters
# model.compile(loss = 'binary_crossentropy',          # Model Compiling   
#               optimizer = Adam(lr = LR),
#               metrics = ['binary_accuracy'])


## Dice Parameters
model.compile(loss = [dice_coef_loss],          # Model Compiling   
              optimizer = Adam(lr = LR),
              metrics = [dice_coef])

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 240, 240, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_209 (Conv2D)            (None, 240, 240, 16  160         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_198 (Batch  (None, 240, 240, 16  64         ['conv2d_209[0][0]']             
 Normalization)                 )                                                          

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
## Get paths for images and masks
datapath = '/DL_course_data/Lab3/MRI/' 
image_path = '/DL_course_data/Lab3/MRI/Image'
mask_path = '/DL_course_data/Lab3/MRI/Mask'

image_path_list = os.listdir(image_path)
mask_path_list = os.listdir(mask_path)

In [146]:
## Try out with flow from dataframe

image_list = [os.path.join(image_path,i) for i in image_path_list]
mask_list = [ i.replace(".png","_Tumor.png") for i in image_list]
data = pd.DataFrame()
data['images'] = image_list
data['masks'] = mask_list


## Generator
datagen = ImageDataGenerator(rescale=1/255.,validation_split=0.2)
    
train_generator = datagen.flow_from_dataframe(
    data,
    x_col = 'images',
    y_col = 'masks',
    color_mode = 'grayscale',
    target_size=(128,128),
    class_model = None,
    batch_size = batch_size,
    subset = 'training',
    shuffle = True,
    seed = 1)

val_generator = datagen.flow_from_dataframe(
    data,
    x_col = 'images',
    y_col = 'masks',
    color_mode = 'grayscale',
    target_size=(128,128),
    class_model = None,
    batch_size = batch_size,
    subset = 'validation',
    shuffle = True,
    seed = 1) 


train_steps = train_generator.n//train_generator.batch_size
val_steps = val_generator.n//val_generator.batch_size

# model_histogram = model.fit_generator(train_generator, 
#     steps_per_epoch = train_steps,
#     validation_data = val_generator, validation_steps = val_steps,
#     epochs = epochs, verbose=1)

Found 7523 validated image filenames belonging to 9403 classes.
Found 1880 validated image filenames belonging to 9403 classes.


In [144]:
def load_data(image_path,mask_path, img_h, img_w, p):
    
    images = []
    masks = []

    for image in image_list:
        img = imread(image, as_gray=True)  # "as_grey"
        img = resize(img, (img_h, img_w), anti_aliasing=True).astype('float32')
        images.append(img)

    for mask in mask_list:
        mask_img = imread(mask, as_gray=True)
        mask = resize(mask_img, (img_h, img_w), anti_aliasing=True).astype('float32')
        masks.append(mask)

    ## Load data in traditional way
    img_train, img_val, mask_train, mask_val = train_test_split(images, masks, shuffle = True,
                                                      test_size = p)
    img_train = np.expand_dims(img_train, axis = -1)
    img_train = np.array(img_train)
    img_val = np.expand_dims(img_val, axis = -1)    
    img_val = np.array(img_val)
    mask_train = np.expand_dims(mask_train, axis = -1)
    mask_train = np.array(mask_train)
    mask_val = np.expand_dims(mask_val, axis = -1)
    mask_val = np.array(mask_val)
    
    return img_train, img_val, mask_train, mask_val

In [147]:
image_list = [os.path.join(image_path,i) for i in image_path_list]
mask_list = [ os.path.join(mask_path,i.replace(".png","_Tumor.png")) for i in image_path_list]
img_train, img_val, mask_train, mask_val = load_data(image_list,mask_list, 240, 240, 0.2)

In [148]:
## Add data augmentation
image_aug = ImageDataGenerator(rotation_range=20,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                zoom_range = 0.3,
                                horizontal_flip=True,
                                rescale=1. / 255)

train_generator = image_aug.flow(
                                img_train,mask_train,
                                batch_size = batch_size)

val_generator = image_aug.flow(
                                img_val,mask_val,
                                batch_size = batch_size)

model_histogram = model.fit_generator(train_generator, steps_per_epoch = train_generator.n//batch_size,
    validation_data = val_generator, validation_steps = val_generator.n//batch_size,
    epochs = epochs,  verbose=1)

Epoch 1/50


<ipython-input-148-c0b37f3fbfff>:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_histogram = model.fit_generator(train_generator, steps_per_epoch = train_generator.n//batch_size,


940/940 [==============================] - 83s 86ms/step - loss: 0.9218 - dice_coef: 0.0782 - val_loss: 0.9323 - val_dice_coef: 0.0677
Epoch 2/50
940/940 [==============================] - 80s 85ms/step - loss: 0.8803 - dice_coef: 0.1197 - val_loss: 0.9857 - val_dice_coef: 0.0143
Epoch 3/50
940/940 [==============================] - 76s 81ms/step - loss: 0.8421 - dice_coef: 0.1578 - val_loss: 0.9889 - val_dice_coef: 0.0111
Epoch 4/50
940/940 [==============================] - 76s 81ms/step - loss: 0.8123 - dice_coef: 0.1875 - val_loss: 0.9968 - val_dice_coef: 0.0032
Epoch 5/50
940/940 [==============================] - 76s 81ms/step - loss: 0.7903 - dice_coef: 0.2098 - val_loss: 0.9966 - val_dice_coef: 0.0034
Epoch 6/50
940/940 [==============================] - 76s 81ms/step - loss: 0.7867 - dice_coef: 0.2132 - val_loss: 0.9988 - val_dice_coef: 0.0012
Epoch 7/50
940/940 [==============================] - 76s 81ms/step - loss: 0.7723 - dice_coef: 0.2276 - val_loss: 0.9995 - val_dice_co

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



747/940 [======================>.......] - ETA: 14s - loss: 0.7486 - dice_coef: 0.2512

KeyboardInterrupt: 

In [ ]:
image = imread('/DL_course_data/Lab3/CT/Image/Im68_54.png')
mask = imread('/DL_course_data/Lab3/CT/Mask/Im68_54.png')

row, col = image.shape
def show_paired(pic_1, pic_2):
    fig, axes = plt.subplots(nrows=1, ncols=2)
    ax = axes.ravel()
    ax[0].imshow(pic_1, cmap='gray')
    ax[0].set_title("Brain image")
    ax[1].imshow(pic_2, cmap='gray')
    ax[1].set_title("Mask image")
    
    plt.tight_layout()
    plt.show()
show_paired(image,mask)

# mask_n = imread('/DL_course_data/Lab3/MRI/Mask/Brats17_TCIA_280_1_t1ce_52_Tumor.png')
# mask_n